# Тестовое задание на позицию DS'а

## Описание датасета

В этом задании предлагается классифицировать сообщения на предмет того, из какого чата они были взяты. Датасет представляет собой сообщения участников двух публичных чатов:
* Чат по Python (label=0)
* Чат по Data Science (label=1)

## Описание задачи

Ваша задача:
1. Сделать предобработку текстов
2. Предложить модель классификации, метрики.
3. Объяснить их выбор (достоинства/недостатки). 
4. Сделать выводы по работе модели(ей) (достоинства/недостатки), а также проблемы, с которыми вы или ваша модель столкнулась.

Было бы интересно посмотреть не только на финальное решение, но и на эксперименты (даже провальные). 



Итоговую модель далее нужно:
1. Обернуть в класс
2. Развернуть модель как REST API
3. Предоставить инструкцию что нужно сделать чтобы развернуть апи модели у себя на компьютере 

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np


In [2]:
pd.set_option('display.max_colwidth', -1)

df = pd.read_csv('train.csv')
df.sample(5)

/var/folders/55/bhwblzw50zq_dtfl9p7kdbq00000gn/T/ipykernel_2839/4054579702.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,text,label
11181,"Какая разница? Я бы тогда больше занимался АСМ или тем же ML, имел бы свободное время, мог бы выбирать среди большего количества направлений, а не так, что я гребу на галере, а потом по ночам решаю mlbootcamp",1
4706,а в nginx для каждой страницы нужно прописывать location?,0
9532,"Я хозяин i3, поэтому у меня одна модель, а не ансамбль, и я оставляю GridSearch на ночь)",1
11846,ну если ты вундеркиндом был я за тебя рад. А вообще один из кор разработиков говорил что что-то будут менять с докой,0
32,"Так у тебя новый не знает о старых, для него ID пойдут с нуля",1


In [3]:
df.shape

(12404, 2)

# Ваше решение

In [4]:
# ! pip3 install nltk
# ! pip3 install pymorphy2
# ! pip3 install stop-words
# ! pip3 install pymystem3
# ! pip3 install catboost
# ! pip3 install gensim
# ! pip3 install flask
# ! pip3 install flask_restful

## Предобработка

In [5]:
import ssl
_create_unverified_https_context = ssl._create_unverified_context
ssl._create_default_https_context = _create_unverified_https_context

import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /Users/ilya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# заготовим стоп-слова

from nltk.corpus import stopwords
from stop_words import get_stop_words

russian_stopwords = stopwords.words('russian')
russian_stopwords_2 = get_stop_words('russian')

russian_stopwords  = list(set(russian_stopwords) | set(russian_stopwords_2))

In [7]:
from string import punctuation

# удалим знаки пунктуации и числа
deleted_symols = punctuation + '0123456789'
func = lambda text: ''.join([char for char in text if str(char) not in deleted_symols])
df['text_prepared'] = df['text'].apply(func)

# удалим стоп-слова
func = lambda text: ' '.join([word for word in text.split(' ') if word not in russian_stopwords])
df['text_prepared'] = df['text_prepared'].str.lower().apply(func)

In [8]:
df.head()

,text,label,text_prepared
0,"если у тебя нет верхнего предела, фиксед поинт это проблема",1,верхнего предела фиксед поинт проблема
1,Ага или паркинг пула подобных сайтов и сравнение цен и др параметров,0,ага паркинг пула подобных сайтов сравнение цен др параметров
2,"я думаю, что скор должен быть близок к максимуму.",1,думаю скор должен близок максимуму
3,Ты делаешь ту задачу с парсингом 20к страниц?) без прокси и без многопоточности уже бы спарсил),0,делаешь задачу парсингом страниц прокси многопоточности спарсил
4,"То есть ты больше веришь, что я реально могу попасть в топ, чем в то, что рандом сейчас лучше моделей?)",1,веришь реально могу попасть топ рандом моделей


In [9]:
# from pymorphy2 import MorphAnalyzer

# # лематизируем слова
# all_word_str = " ".join(df["text_prepared"])
# all_word_list = all_word_str.split()
# all_unique_word = np.unique(all_word_list)
# lemmatized_word_dict = {}
# lemmatizer = MorphAnalyzer()

# for word in all_unique_word:
#     lemmatized_word_dict[word] = lemmatizer.normal_forms(word)[0]
# func = lambda text: ' '.join([lemmatized_word_dict[word] for word in text.split()])


# df['text_prepared'] = df['text_prepared'].apply(func)



## смотрел рандомные примеры: местами не все глаголы лематизировались, особенно англицизимы
## попробовал другой, стало получше

In [10]:
from pymystem3 import Mystem

mystem = Mystem() 
func = lambda text: ''.join(mystem.lemmatize(text)).strip()
df['text_prepared'] = df['text_prepared'].apply(func)

In [11]:
df.head()

,text,label,text_prepared
0,"если у тебя нет верхнего предела, фиксед поинт это проблема",1,верхний предел фикседой поинт проблема
1,Ага или паркинг пула подобных сайтов и сравнение цен и др параметров,0,ага паркинг пул подобный сайт сравнение цена др параметр
2,"я думаю, что скор должен быть близок к максимуму.",1,думать скорый должный близкий максимум
3,Ты делаешь ту задачу с парсингом 20к страниц?) без прокси и без многопоточности уже бы спарсил),0,делать задача парсинг страница прокси многопоточность спарсить
4,"То есть ты больше веришь, что я реально могу попасть в топ, чем в то, что рандом сейчас лучше моделей?)",1,верить реально мочь попадать топ ранд модель


## Векторизация текста

In [12]:
from sklearn.model_selection import train_test_split

X = df['text_prepared']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, 
                                                    test_size=0.2, random_state=0)


### TF-IDF

In [13]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# начнем с классики
vectorizer = TfidfVectorizer(
    min_df=10, 
    max_df=50, 
    ngram_range=(1, 5)
)

vectorizer.fit(X_train)
X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)

X_train_vec.shape

(9923, 1166)

### word2vec

In [14]:
from gensim.models import Word2Vec

# попробуем чуть более-продвинутую векторизацию текста
# эмбеддинги слов просто усредним
docs = [doc.split() for doc in X_train]
w2v = Word2Vec(sentences=docs, vector_size=100, window=4, sg=0, min_count=1, epochs=100, seed=0)


In [15]:
def w2v_trans_word(word, w2v=w2v):
    '''
    вспомогательная функция для обработки слов, которых нет в словаре
    '''
    try:
        return w2v.wv.get_vector(word)
    except:
        return np.zeros(100)

def w2v_trans_doc(doc, w2v=w2v):
    '''
    функция для преобразования pd.Series в вектор w2v через метод apply
    '''
    return np.mean([w2v_trans_word(word, w2v) for word in doc.split()], axis=0)
    
def w2v_vectorizer(X_test, w2v=w2v):
    '''
    функция для преобразования pd.Series в вектор w2v через метод apply,
    а также для обработки ошибок сообщений, где оказались только стоп-слова
    '''
    return np.vstack([np.zeros(100) if np.isnan(np.std(doc)) else doc for doc in 
                     X_test.apply(w2v_trans_doc, w2v).tolist()])


In [16]:
X_train_w2v = w2v_vectorizer(X_train)
X_test_w2v = w2v_vectorizer(X_test)

X_train_w2v.shape

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


(9923, 100)

### doc2vec

In [17]:
from gensim.models import doc2vec

# попробуем векторизацю документов целиком
# усреднение после w2v может вызвать сильные искажения для длинных постов
def tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield doc2vec.TaggedDocument(list_of_words, [i])

data_for_training = list(tagged_document(docs))
d2v_model = doc2vec.Doc2Vec(vector_size=100, min_count=1, epochs=100, seed=0)
d2v_model.build_vocab(data_for_training)
d2v_model.train(data_for_training, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)
  

In [18]:
X_train_d2v = np.stack([d2v_model.infer_vector(doc.split()) for doc in X_train])
X_test_d2v = np.stack([d2v_model.infer_vector(doc.split()) for doc in X_test])
X_train_d2v.shape

(9923, 100)

# Модели

## LogisticRegression

In [19]:
from sklearn.linear_model import LogisticRegression


clf = LogisticRegression(max_iter=3000, random_state=0) #, penalty='l1', C=1, solver='saga')
# clf.fit(X_train, y_train)

## чуть-чуть играл с отсевом признаков, инсайтов не было особо
# k = clf.coef_.shape[1]
# k_zero = (clf.coef_==0).sum()
# print(f'{k_zero} из {k} коэффициентов занунилось')
# print(f'осталось коэффициентов {k-k_zero}')

In [20]:
Xs = [
    [X_train_vec, X_test_vec],
    [X_train_w2v, X_test_w2v],
    [X_train_d2v, X_test_d2v],
]


In [21]:
from sklearn.metrics import classification_report, accuracy_score

In [22]:
acc = []
clfs = {}
for i, preprocess in enumerate(Xs):
    clf.fit(Xs[i][0], y_train)
    clfs[i] = clf
    acc.append([
        accuracy_score(y_train, clf.predict(Xs[i][0])),
        accuracy_score(y_test, clf.predict(Xs[i][1]))
        ])
    
# результаты лог регрессии

acc_clf = pd.DataFrame(acc, index=['TF-IDF', 'word2vec', 'doc2vec'], columns=['train', 'test'])
acc_clf

,train,test
TF-IDF,0.775773,0.729141
word2vec,0.762068,0.762999
doc2vec,0.724781,0.737606


## KNN

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler

knn = KNeighborsClassifier(n_neighbors=30)
scaler = StandardScaler(with_mean=False)

In [24]:
acc = []
for i, preprocess in enumerate(Xs):
    scaler.fit(Xs[i][0])
    knn.fit(scaler.transform(Xs[i][0]), y_train)
    acc.append([
        accuracy_score(y_train, knn.predict(scaler.transform(Xs[i][0]))),
        accuracy_score(y_test, knn.predict(scaler.transform(Xs[i][1])))
        ])
    
# результаты KNN

acc_knn = pd.DataFrame(acc, index=['TF-IDF', 'word2vec', 'doc2vec'], columns=['train', 'test'])
acc_knn

,train,test
TF-IDF,0.673687,0.641677
word2vec,0.761161,0.740830
doc2vec,0.653633,0.659815


## CatBoostClassifier

In [25]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier(metric_period=500, random_state=0)

In [26]:
acc = []
cats = {}
for i, preprocess in enumerate(Xs):
    cat.fit(Xs[i][0], y_train)
    cats[i] = cat
    acc.append([
        accuracy_score(y_train, cat.predict(Xs[i][0])),
        accuracy_score(y_test, cat.predict(Xs[i][1]))
        ])
    
# результаты катбуста

acc_cat = pd.DataFrame(acc, index=['TF-IDF', 'word2vec', 'doc2vec'], columns=['train', 'test'])
acc_cat

Learning rate set to 0.027447
0:	learn: 0.6923653	total: 66.2ms	remaining: 1m 6s
500:	learn: 0.5798352	total: 4.54s	remaining: 4.53s
999:	learn: 0.5269313	total: 9.09s	remaining: 0us
Learning rate set to 0.027447
0:	learn: 0.6869225	total: 8.69ms	remaining: 8.68s
500:	learn: 0.3777419	total: 4.83s	remaining: 4.82s
999:	learn: 0.2714884	total: 9.49s	remaining: 0us
Learning rate set to 0.027447
0:	learn: 0.6905108	total: 9.02ms	remaining: 9.01s
500:	learn: 0.4465834	total: 4.55s	remaining: 4.53s
999:	learn: 0.3302244	total: 9.11s	remaining: 0us


,train,test
TF-IDF,0.763580,0.706973
word2vec,0.924720,0.778315
doc2vec,0.917364,0.732769


In [27]:
print(classification_report(y_test, cat.predict(Xs[i][1])))

              precision    recall  f1-score   support

           0       0.71      0.69      0.70      1104
           1       0.75      0.77      0.76      1377

    accuracy                           0.73      2481
   macro avg       0.73      0.73      0.73      2481
weighted avg       0.73      0.73      0.73      2481



# Результаты

Для начала был выбран простой классический подход, а именно TF-IDF для векторизации текста в связке с лог. регрессией. Уже он показал неплохое качество, однако далее была попытка его улучшить. Для векторизации были сделаны эмбеддинги с помощью word2vec, а затем и с помощью doc2vec. А в качестве альтернативных моделей были взяты KNN и градиентный бустинг. KNN выбран из той логики, что на стандартизованных эмбеддингах он может себя хорошо показать, а бустинг в целом часто хорошо себя показывает.

В качестве метрики выбрана accuracy, поскольку классы сбалансированы. Полнота и точность по обоим классам была в целом сопоставима, поэтому accuracy для анализа достаточно. Пример с полнотой и точностью приведен для тестовой выборки у бустинга (по остальным экспериментам была похожая динамика).

Как показали эксперименты, эмбеддинги word2vec оказались лучше эмбеддингов doc2vec, векторизация с помощью TF-IDF оказалась хуже всех. Динамика была схожа для всех моделей. Лучше всего себя показал себя бустинг, но лог. регрессия всего немного хуже, метод ближ. соседей значительно хуже обеих моделей.

В качестве итоговой тем не менее выбрана лог. регрессия за счёт простоты и скорости предикта. Для векторизации взят word2vec.

Ниже пара примеров с тем, как делается предсказание.

---

Для запуска api нужно установить requirements.

Команда чтобы запустить апи: python api.py

Деплой тут: http://127.0.0.1:5000/

Примеры, чтобы потестить
1. переменные аргументы функции класс лямбда
2. логрег бустинг эмбединнги топ

In [28]:
results = pd.concat([acc_clf,acc_knn,acc_cat], keys=['LR', 'KNN', 'CB'], axis=1)
results

LR                 KNN                  CB          
             train      test     train      test     train      test
TF-IDF    0.775773  0.729141  0.673687  0.641677  0.763580  0.706973
word2vec  0.762068  0.762999  0.761161  0.740830  0.924720  0.778315
doc2vec   0.724781  0.737606  0.653633  0.659815  0.917364  0.732769

In [29]:
# сохраним лог. рег
final = clfs[1]

In [30]:
import pickle

with open('model.pickle', 'wb') as f:
    pickle.dump(final, f)
    
with open('w2v.pickle', 'wb') as f:
    pickle.dump(w2v, f)
    


## Тест

In [31]:
import pickle

with open('model.pickle', 'rb') as f:
    model = pickle.load(f)
    
with open('w2v.pickle', 'rb') as f:
    vectorizer = pickle.load(f)
    


In [32]:
from api import prepare


def get_pred(doc, w2v=w2v):
    vec = w2v_vectorizer(pd.Series(doc), w2v)
    proba = model.predict_proba(vec)[:,1][0]

    if proba>0.5:
        return f'Это чат по Data Science с вероятностью {round(proba*100)}% : \t {doc}'
    else:
        return f'Это чат по Python с вероятностью {round((1-proba)*100)}% : \t\t {doc}'
        

        

In [33]:
X_test_simple = pd.Series([
    'переменные аргументы функции класс лямбда',
    'логрег бустинг эмбединнги топ'
])

In [34]:
for x in X_test_simple:
    print(get_pred(prepare(x)))

Это чат по Python с вероятностью 100% : 		 переменный аргумент функция класс лямбда
Это чат по Data Science с вероятностью 100% : 	 логрег бустинг эмбединнга топ


In [35]:
# ! pip3 freeze > requirements.txt